In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, classification_report
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
import pandas as pd
dataset = pd.read_csv("/content/drive/My Drive/Deep Learning/pima-indians-diabetes-database/diabetes.csv")

In [3]:
dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [0]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop('Outcome', axis=1), dataset[['Outcome']], test_size=0.2, random_state=1994)

In [5]:
# creating tensor from pandas 
train_tensor_input = torch.tensor(x_train.values.astype(np.float32))
train_tensor_output = torch.tensor(y_train.values.astype(np.float32))
test_tensor_input = torch.tensor(x_test.values.astype(np.float32))
test_tensor_output = torch.tensor(y_test.values.astype(np.float32))
# printing output tensor result
print(train_tensor_input.shape, train_tensor_output.shape, test_tensor_input.shape, test_tensor_output.shape)

torch.Size([614, 8]) torch.Size([614, 1]) torch.Size([154, 8]) torch.Size([154, 1])


In [0]:
batch_size = 10
#Taking Data from Tensor into DataLoader Function pytorch
train_tensor = TensorDataset(train_tensor_input, train_tensor_output) 
train_loader = DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

#Taking Data from Tensor into DataLoader Function pytorch
test_tensor = TensorDataset(test_tensor_input, test_tensor_output) 
test_loader = DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)

In [7]:
#Model
class ANN(nn.Module):
  def __init__(self):
    super().__init__()

    #Inputs to hidden layer 1 linear transformation
    self.hidden1 = nn.Linear(8, 13)
    #Inputs to hidden layer 2 linear transformation
    self.hidden2 = nn.Linear(13, 10)
    #Inputs to hidden layer 3 linear transformation
    self.hidden3 = nn.Linear(10, 7)
    #Output Layer 10 units - One for each digit.
    self.output = nn.Linear(7, 1)


  def forward(self, x):
    #pass the input tensor to each of the operations.
    x = self.hidden1(x)
    x = F.relu(x)
    x = self.hidden2(x)
    x = F.relu(x)
    x = self.hidden3(x)
    x = F.relu(x)
    x = self.output(x)
    x = torch.sigmoid(x)
    return x
MLP = ANN()
print(MLP)
#Loss Criterion
loss_criterion = nn.BCELoss()
#Optimizer 
optimizer = optim.Adam(MLP.parameters(), lr=0.0001)

ANN(
  (hidden1): Linear(in_features=8, out_features=13, bias=True)
  (hidden2): Linear(in_features=13, out_features=10, bias=True)
  (hidden3): Linear(in_features=10, out_features=7, bias=True)
  (output): Linear(in_features=7, out_features=1, bias=True)
)


In [14]:
epochs = 1000
lowest_validation_loss = None
model_save_path = '/content/drive/My Drive/Pytorch/MLP.pt'
for e in range(epochs):
  running_loss = 0
  for train_inputs, train_labels in train_loader:
    #Setting the Gradients to Zero
    optimizer.zero_grad()
    #Forward Pass
    train_outputs = MLP(train_inputs)
    #Loss Calculation
    loss = loss_criterion(train_outputs, train_labels)
    #Backward Propagation
    loss.backward()
    #Updating Gradients
    optimizer.step()
    #Cost Calculation
    running_loss+=loss.item()
  else:
    with torch.no_grad():
      validation_loss = loss_criterion(MLP(test_tensor_input), test_tensor_output)
      if (lowest_validation_loss is None) or (validation_loss < lowest_validation_loss):
        lowest_validation_loss = validation_loss.item()
        #Saving Model weigths with lowest validation loss
        print(f'Epoch {e} Training Loss: {running_loss/(len(train_loader))}, Validation Loss: {validation_loss}')
        torch.save(MLP.state_dict(), model_save_path)
    if e%10==0:
      print(f'Epoch {e} Training Loss: {running_loss/(len(train_loader))}, Validation Loss: {validation_loss}')

Epoch 0 Training Loss: 0.5568173546944896, Validation Loss: 0.6412449479103088
Epoch 0 Training Loss: 0.5568173546944896, Validation Loss: 0.6412449479103088
Epoch 1 Training Loss: 0.552263508881292, Validation Loss: 0.640596866607666
Epoch 3 Training Loss: 0.5533356729053682, Validation Loss: 0.6397342085838318
Epoch 4 Training Loss: 0.5488434650244252, Validation Loss: 0.6392588019371033
Epoch 5 Training Loss: 0.5508482162990878, Validation Loss: 0.6389068365097046
Epoch 6 Training Loss: 0.550665317523864, Validation Loss: 0.6387301087379456
Epoch 8 Training Loss: 0.5460995195373413, Validation Loss: 0.6363465189933777
Epoch 10 Training Loss: 0.54779392625055, Validation Loss: 0.6380131840705872
Epoch 16 Training Loss: 0.5455033250393406, Validation Loss: 0.6360687613487244
Epoch 20 Training Loss: 0.5422039176187208, Validation Loss: 0.6399686336517334
Epoch 27 Training Loss: 0.5424612885521304, Validation Loss: 0.6356171369552612
Epoch 30 Training Loss: 0.5417568015475427, Validatio

In [15]:
#Load the saved model path
MLP.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [16]:
with torch.no_grad():
  prediction = MLP(test_tensor_input)
  print(roc_auc_score(test_tensor_output, prediction))
  #Threshold Evaluator
  prediction[prediction>0.5]=1
  prediction[prediction<=0.5]=0
  print(accuracy_score(test_tensor_output, prediction))
  print(classification_report(test_tensor_output, prediction))

0.7305976806422837
0.6948051948051948
              precision    recall  f1-score   support

         0.0       0.72      0.82      0.77        95
         1.0       0.63      0.49      0.55        59

    accuracy                           0.69       154
   macro avg       0.68      0.66      0.66       154
weighted avg       0.69      0.69      0.69       154



In [17]:
#Keras Model to Compare
# Create first network with Keras
from keras.models import Sequential
from keras.layers import Dense
# create model
model = Sequential()
model.add(Dense(13,input_dim=8,init='uniform',activation='relu'))
model.add(Dense(10,init='uniform',activation='relu'))
model.add(Dense(7,init='uniform',activation='relu'))
model.add(Dense(1,init='uniform',activation='sigmoid'))
#Compile model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#Fit the model
model.fit(x_train,y_train,nb_epoch=1000,batch_size=10, validation_data=[x_test, y_test])

Using TensorFlow backend.
